<a href="https://colab.research.google.com/github/Francho03/Praca-licencjacka/blob/main/Agregowanie_danych_do_powiatow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np


file_path = "/content/drive/MyDrive/praca_licencjacka/Sprawozdanie_przemoc_2014_2023.xlsx"
columns = ["lp", "woj", "pow", "gm", "typ_gm", "typ_jednostki", "keso", "rok",
           "liczba_proc_nk_a", "liczba_proc_nk_c", "rodziny_wszczete",
           "liczba_osob_ogol", "liczba_osob_kobiety", "liczba_osob_mezczyzni",
           "liczba_osob_dzieci"]

dane = pd.read_excel(file_path, names=columns, engine='openpyxl')


dane["keso"] = dane["keso"].astype(str)
dane["keso"] = dane["keso"].str.extract(r"(\d{12})")[0].str.zfill(12)


dane["jednostka_typ"] = dane["keso"].str[0:2]
dane["wojew_kod"] = dane["keso"].str[2:4]
dane["powiat_kod"] = dane["keso"].str[4:6]
dane["woj_pow_kod"] = dane["wojew_kod"] + dane["powiat_kod"]

valid_dane = dane[
    dane["keso"].str.match(r"^\d{12}$", na=False) &
    (dane["powiat_kod"] != "00") &
    (dane["woj"].notna())
].copy()


numeric_cols = [
    "liczba_proc_nk_a", "liczba_proc_nk_c", "rodziny_wszczete",
    "liczba_osob_ogol", "liczba_osob_kobiety",
    "liczba_osob_mezczyzni", "liczba_osob_dzieci"
]

for col in numeric_cols:
    valid_dane[col] = pd.to_numeric(valid_dane[col], errors='coerce')


results = []
years = valid_dane["rok"].unique()

for year in years:
    year_data = valid_dane[valid_dane["rok"] == year].copy()


    grouped = year_data.groupby(["woj", "pow", "woj_pow_kod", "rok"])


    agg_data = grouped[numeric_cols].sum(min_count=1)

    meta_data = grouped[["keso", "wojew_kod", "powiat_kod"]].first()

    result = pd.concat([meta_data, agg_data], axis=1).reset_index()
    results.append(result)


final_result = pd.concat(results, ignore_index=True)


,woj,pow,woj_pow_kod,rok,keso,wojew_kod,powiat_kod,liczba_proc_nk_a,liczba_proc_nk_c,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
3725,zachodniopomorskie,m. Szczecin,3262,2014,253262011000,32,62,78,1296.0,1296.0,0.0,NaN,NaN,NaN
1933,dolnośląskie,średzki,0218,2018,300218012000,02,18,7,42.0,57.0,200.0,118.0,39.0,43.0
1425,wielkopolskie,wągrowiecki,3028,2020,303028022000,30,28,35,74.0,168.0,519.0,191.0,74.0,254.0
3506,lubelskie,świdnicki,0617,2014,300617022000,06,17,37,142.0,173.0,290.0,NaN,NaN,NaN
979,pomorskie,lęborski,2208,2021,302208032000,22,08,49,125.0,184.0,250.0,165.0,25.0,60.0
62,lubelskie,lubelski,0609,2023,300609013000,06,09,57,198.0,390.0,571.0,286.0,51.0,234.0
3409,śląskie,raciborski,2411,2015,302411022000,24,11,19,248.0,182.0,330.0,260.0,12.0,58.0
2447,opolskie,m. Opole,1661,2017,251661011000,16,61,53,181.0,198.0,24.0,17.0,7.0,0.0
3377,łódzkie,łódzki wschodni,1006,2015,301006022000,10,06,15,107.0,141.0,222.0,209.0,8.0,5.0
3468,kujawsko - pomorskie,m. Grudziądz,0462,2014,250462011000,04,62,30,131.0,284.0,0.0,NaN,NaN,NaN


In [12]:
Sprawozdanie_2014_2023_zagregowane = final_result.drop(["woj_pow_kod", "wojew_kod", "powiat_kod"],axis=1)



In [13]:
Sprawozdanie_2014_2023_zagregowane

,woj,pow,rok,keso,liczba_proc_nk_a,liczba_proc_nk_c,rodziny_wszczete,liczba_osob_ogol,liczba_osob_kobiety,liczba_osob_mezczyzni,liczba_osob_dzieci
0,dolnośląskie,bolesławiecki,2023,300201011000,42,181.0,383.0,493.0,294.0,95.0,104.0
1,dolnośląskie,dzierżoniowski,2023,300202011000,65,132.0,291.0,402.0,216.0,26.0,160.0
2,dolnośląskie,górowski,2023,300204013000,23,56.0,125.0,273.0,211.0,23.0,39.0
3,dolnośląskie,głogowski,2023,200203000000,36,199.0,428.0,770.0,333.0,72.0,365.0
4,dolnośląskie,jaworski,2023,300205023000,12,59.0,119.0,123.0,93.0,8.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...
3805,świętokrzyskie,sandomierski,2014,302609022000,8,220.0,145.0,196.0,NaN,NaN,NaN
3806,świętokrzyskie,skarżyski,2014,302610022000,32,62.0,166.0,215.0,NaN,NaN,NaN
3807,świętokrzyskie,starachowicki,2014,302611022000,4,80.0,218.0,680.0,NaN,NaN,NaN
3808,świętokrzyskie,staszowski,2014,302612012000,21,199.0,216.0,449.0,NaN,NaN,NaN


In [14]:
Sprawozdanie_2014_2023_zagregowane.to_excel("/content/drive/MyDrive/praca_licencjacka/Sprawozdanie_2014_2023_zagregowane.xlsx", sheet_name= "Sprawozdanie_2014_2023_zagregowane" ,index=False)

/usr/local/lib/python3.11/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [16]:
with pd.ExcelWriter('Sprawozdanie_2014_2023_zagregowane.xlsx') as writer:
 Sprawozdanie_2014_2023_zagregowane.to_excel(writer, index=False, sheet_name='Sprawozdanie_2014_2023_zagregowane')